In [226]:
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import chain
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage



embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash",temperature=0.3, max_tokens=1000)


# general information
url_list = [
    "https://apnews.com/article/sri-lanka-presidential-election-explained-wickremesinghe-e203abecd36d04fde21e927b9be8ba71",
    "https://www.electionguide.org/elections/id/4364/",
    "https://thediplomat.com/2024/08/sri-lankas-anura-dissanayake-a-strong-contender-for-president/",
    "https://www.isas.nus.edu.sg/papers/sri-lanka-election-2024-key-political-candidates-and-campaigns/",
    "https://www.thehindu.com/news/international/in-southern-sri-lanka-a-chant-for-change-this-poll-season/article68628670.ece",
    "https://groundviews.org/2024/09/04/the-hard-truth-supporting-ranil-now-could-trigger-the-next-crisis/",
    "https://groundviews.org/2024/08/29/the-jvp-and-ethnic-relations-walking-a-tightrope-to-2024-part-1/",
    "https://groundviews.org/2024/09/10/changes-in-support-for-the-leading-presidential-candidates/",
    "https://groundviews.org/2024/09/13/voters-want-more-government-spending-on-health-and-education/",
    "https://groundviews.org/2024/07/18/a-k-dissanayake-continues-to-lead-in-favourability-ratings/",
    "https://groundviews.org/2024/07/22/the-three-way-contest-to-win-the-presidency/",
    "https://www.vifindia.org/article/2024/september/05/Sri-Lanka-at-the-Crossroads-The-Presidential-Election-2024",
    "https://www.dailymirror.lk/breaking-news/Anura-rebuffs-Ranils-proposal-to-invite-IMF-for-debate/108-291616"
]
general_loader = WebBaseLoader(url_list)
general_data = general_loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
general_docs = text_splitter.split_documents(general_data)
vectorstore_general = Chroma.from_documents(general_docs, embeddings, collection_name="general")
retriever_general = vectorstore_general.as_retriever(search_kwargs={"k": 10})


# sajith's manifesto
sajith_loader = PyPDFLoader("Sajith_manifesto_english.pdf")
sajith_data = sajith_loader.load()
sajith_docs = text_splitter.split_documents(sajith_data)
vectorstore_sajith = Chroma.from_documents(sajith_docs, embeddings, collection_name="sajith_premadasa")
retriever_sajith = vectorstore_sajith.as_retriever(search_kwargs={"k": 10})

# akd's manifesto
akd_loader = PyPDFLoader("AKD Manifesto.pdf")
akd_data = akd_loader.load()
akd_docs = text_splitter.split_documents(akd_data)
vectorstore_akd = Chroma.from_documents(akd_docs, embeddings, collection_name="anura_kumara_dissanayake")
retriever_akd = vectorstore_akd.as_retriever(search_kwargs={"k": 10})

# ranil's manifesto
ranil_loader = PyPDFLoader("ranil_manifesto_compressed-output.pdf")
ranil_data = ranil_loader.load()
ranil_docs = text_splitter.split_documents(ranil_data)
vectorstore_ranil = Chroma.from_documents(ranil_docs, embeddings, collection_name="ranil_wickramasinghe")
retriever_ranil = vectorstore_ranil.as_retriever(search_kwargs={"k": 10})

In [158]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

history_aware_retriever_general = create_history_aware_retriever(
    llm, retriever_general, contextualize_q_prompt
)

history_aware_retriever_sajith = create_history_aware_retriever(
    llm, retriever_sajith, contextualize_q_prompt
)

history_aware_retriever_akd = create_history_aware_retriever(
    llm, retriever_akd, contextualize_q_prompt
)

history_aware_retriever_ranil = create_history_aware_retriever(
    llm, retriever_ranil, contextualize_q_prompt
)

In [211]:
retrievers = {
    "sajith_premadasa": history_aware_retriever_sajith,
    "anura_kumara_dissanayake": history_aware_retriever_akd,
    "ranil_wickramasinghe": history_aware_retriever_ranil,
}

chat_history = []

class SearchAndCompare(BaseModel):
    """Search for information about a person or compare informations about persons."""

    queryType: str = Field(
        ...,
        description="Query type. Should be `search` or `compare` or `general`. if there's only one person name it's search, if there are many person's name it's compare, or it can be a general question which does not require any specific person",)

    query: str = Field(
        ...,
        description="Query to look up or query to compare",
    )

    candidates: int = Field(
        ...,
        description="Number of persons to search or compare. can be 0 for general questions",
    )

    person1: str = Field(
        ...,
        description="Person to look things up for or persons to compare. Should be `sajith_premadasa` or `anura_kumara_dissanayake` or `ranil_wickramasinghe` or can be 'null'.",
    )
    person2: str = Field(
        ...,
        description="Person to look things up for or persons to compare. Should be `sajith_premadasa` or `anura_kumara_dissanayake` or `ranil_wickramasinghe` or can be 'null'.",
    )
    person3: str = Field(
        ...,
        description="Person to look things up for or persons to compare. Should be `sajith_premadasa` or `anura_kumara_dissanayake` or `ranil_wickramasinghe` or can be 'null'.",
    )

system_query = """You have the ability to determine whether the user question is general, or it is related to a specific person or it is a comparison between multiple persons. if you can't find the type set it as general"""
prompt_query = ChatPromptTemplate.from_messages(
    [
        ("system", system_query),
        ("human", "{question}"),
    ]
)

structured_llm_query = llm.with_structured_output(SearchAndCompare)
query_analyzer = {"question": RunnablePassthrough()} | prompt_query | structured_llm_query

# @chain
# def query_analyzer(question):
#     # system_query = """You have the ability to determine whether the user question is general, or it is related to a specific person or it is a comparison between multiple persons. and if the question requires chat history use the provided chat history to analyze the query"""
#     contextualize_q_system_prompt = (
#     "Given a chat history and the latest user question "
#     "which might reference context in the chat history, "
#     "formulate a standalone query which can be understood "
#     "without the chat history. Do NOT answer the question, "
#     "just reformulate it if needed and otherwise return it as is."
#     "You have the ability to determine whether the user question is general, or it is related to a specific person or it is a comparison between multiple persons."
# )

#     prompt_query = ChatPromptTemplate.from_messages(
#         [
#             ("system", contextualize_q_system_prompt),
#             MessagesPlaceholder("chat_history"),
#             ("human", "{question}"),
#         ]
#     )
#     structured_llm_query = llm.with_structured_output(SearchAndCompare)
#     prompt_query = prompt_query.format_messages(chat_history=chat_history, question=question)
#     print(prompt_query)
#     response = structured_llm_query.invoke(prompt_query)

#     return response

@chain
def qa_chain(question):
    print("1111")
    response = query_analyzer.invoke(question)
    print(response)
    if response.queryType == "search" or response.queryType == "compare":
        if response.queryType == "search":
            retriever = retrievers[response.person1]
            retrieved_docs = retriever.invoke({"input":response.query, "chat_history": chat_history})

            prompt = (
            "system :"
            "You are an assistant for question-answering tasks. "
            "Use the following pieces of retrieved context to answer "
            "the question. If you don't know the answer, say that you "
            "don't know."
            "\n\n"
            "{context}"
            "\n\n"

            "chat_history :" 
            "{chat_history}"

            "human :"
            "{question}"
            ).format(context=retrieved_docs, question=question, chat_history=chat_history)

            result = llm.invoke(prompt)

            return result
    
        elif response.queryType == "compare":
            retriever1 = retrievers[response.person1]
            retrieved_docs1 = retriever1.invoke({"input":response.query, "chat_history": chat_history})

            if response.person2 != 'null':
                retriever2 = retrievers[response.person2]
                retrieved_docs2 = retriever2.invoke({"input":response.query, "chat_history": chat_history})
            else:
                retrieved_docs2 = ''

            if response.person3 != 'null':
                retriever3 = retrievers[response.person3]
                retrieved_docs3 = retriever3.invoke({"input":response.query, "chat_history": chat_history})
            else:
                retrieved_docs3 = ''

            prompt = (
            "system :"
            "You are an assistant for comparing manifestos. "
            "Use the following pieces of retrieved context from different manifestos to answer "
            "the question. If you don't know the answer, say that you "
            "don't know."
            "\n\n"
            "{context1}"
            "\n\n"
            "{context2}"
            "\n\n"
            "{context3}"
            "\n\n"

            "chat_history :" 
            "{chat_history}"

            "human :"
            "{question}"
            ).format(context1=retrieved_docs1, context2=retrieved_docs2, context3=retrieved_docs3, question=question, chat_history=chat_history)

            result = llm.invoke(prompt)

            return result
    else:
        retriever = history_aware_retriever_general
        retrieved_docs = retriever.invoke({"input":response.query, "chat_history": chat_history})

        prompt = (
            "system :"
            "You are an assistant for question-answering tasks related to srilankan election."
            "Use the following pieces of retrieved context to answer "
            "the question. If you don't know the answer, say that you "
            "don't know."
            "or if the question is not much related to srilankan election say that this question is not related to srilankan election ass a election chatbot i can't provide you with answer this."
            "\n\n"
            "{context}"
            "\n\n"

            "chat_history :" 
            "{chat_history}"

            "human :"
            "{question}"
            ).format(context=retrieved_docs, question=question, chat_history=chat_history)

        result = llm.invoke(prompt)
        return result

from langchain_core.messages import AIMessage, HumanMessage

def chatbot(question):
    result = qa_chain.invoke(question)
    
    chat_history.extend([
            HumanMessage(content=question),
            AIMessage(content=result.content),
        ])
    return result.content

In [8]:
# from langchain_core.messages import AIMessage, HumanMessage

# def chatbot(question):
#     result = qa_chain.invoke(question)
#     chat_history.extend(
#         [
#             HumanMessage(content=question),
#             AIMessage(content=result.content),
#         ]
#     )
#     return result.content

In [212]:
chat_history

[]

In [213]:
question = "what is blue"
print(chatbot(question))

1111
queryType='general' query='what is blue' candidates=0 person1='null' person2='null' person3='null'
This question is not related to Sri Lankan elections. As an election chatbot, I can't provide you with an answer to this. 



In [214]:
chat_history

[HumanMessage(content='what is blue'),
 AIMessage(content="This question is not related to Sri Lankan elections. As an election chatbot, I can't provide you with an answer to this. \n")]

In [215]:
question = "what is 9+1"
print(chatbot(question))

1111
queryType='general' query='what is 9+1' candidates=0 person1='null' person2='null' person3='null'
This question is not related to Sri Lankan elections. As an election chatbot, I can't provide you with an answer to this. 



In [216]:
chat_history

[HumanMessage(content='what is blue'),
 AIMessage(content="This question is not related to Sri Lankan elections. As an election chatbot, I can't provide you with an answer to this. \n"),
 HumanMessage(content='what is 9+1'),
 AIMessage(content="This question is not related to Sri Lankan elections. As an election chatbot, I can't provide you with an answer to this. \n")]

In [217]:
question = "who is sajith"
print(chatbot(question))

1111
queryType='search' query='who is' candidates=1 person1='sajith_premadasa' person2='null' person3='null'
Sajith Premadasa is the leader of the Samagi Jana Sandanaya (SJS) and the candidate for the upcoming Presidential election in Sri Lanka. He is presenting his manifesto, "Win for all," to outline his vision for a positive national transformation. 



In [218]:
chat_history

[HumanMessage(content='what is blue'),
 AIMessage(content="This question is not related to Sri Lankan elections. As an election chatbot, I can't provide you with an answer to this. \n"),
 HumanMessage(content='what is 9+1'),
 AIMessage(content="This question is not related to Sri Lankan elections. As an election chatbot, I can't provide you with an answer to this. \n"),
 HumanMessage(content='who is sajith'),
 AIMessage(content='Sajith Premadasa is the leader of the Samagi Jana Sandanaya (SJS) and the candidate for the upcoming Presidential election in Sri Lanka. He is presenting his manifesto, "Win for all," to outline his vision for a positive national transformation. \n')]

In [219]:
question = "what are his policies"
print(chatbot(question))

1111
queryType='general' query='what are his policies' candidates=0 person1='null' person2='null' person3='null'
Sajith Premadasa's manifesto, "Win for All," focuses on five key points:

* **Building a resilient economy:** This likely includes plans to address the economic crisis, promote growth, and create jobs.
* **Empowering every citizen:** This could involve policies to improve education, healthcare, and social welfare.
* **Enhancing government service:** This might focus on improving efficiency, transparency, and accountability in government.
* **Protecting the quality of life:** This could involve measures to safeguard the environment, ensure food security, and improve public safety.
* **Safeguarding the nation:** This likely includes plans to strengthen national security and protect Sri Lanka's interests. 



In [220]:
chat_history

[HumanMessage(content='what is blue'),
 AIMessage(content="This question is not related to Sri Lankan elections. As an election chatbot, I can't provide you with an answer to this. \n"),
 HumanMessage(content='what is 9+1'),
 AIMessage(content="This question is not related to Sri Lankan elections. As an election chatbot, I can't provide you with an answer to this. \n"),
 HumanMessage(content='who is sajith'),
 AIMessage(content='Sajith Premadasa is the leader of the Samagi Jana Sandanaya (SJS) and the candidate for the upcoming Presidential election in Sri Lanka. He is presenting his manifesto, "Win for all," to outline his vision for a positive national transformation. \n'),
 HumanMessage(content='what are his policies'),
 AIMessage(content='Sajith Premadasa\'s manifesto, "Win for All," focuses on five key points:\n\n* **Building a resilient economy:** This likely includes plans to address the economic crisis, promote growth, and create jobs.\n* **Empowering every citizen:** This coul

In [221]:
question = "who is ranil"
print(chatbot(question))

1111
queryType='general' query='who is ranil wickramasinghe' candidates=0 person1='null' person2='null' person3='null'
Ranil Wickremesinghe is the incumbent President of Sri Lanka and is running for re-election in the upcoming presidential election. He is running as an independent candidate. He replaced ousted President Gotabaya Rajapaksa through a parliamentary vote. 



In [222]:
chat_history

[HumanMessage(content='what is blue'),
 AIMessage(content="This question is not related to Sri Lankan elections. As an election chatbot, I can't provide you with an answer to this. \n"),
 HumanMessage(content='what is 9+1'),
 AIMessage(content="This question is not related to Sri Lankan elections. As an election chatbot, I can't provide you with an answer to this. \n"),
 HumanMessage(content='who is sajith'),
 AIMessage(content='Sajith Premadasa is the leader of the Samagi Jana Sandanaya (SJS) and the candidate for the upcoming Presidential election in Sri Lanka. He is presenting his manifesto, "Win for all," to outline his vision for a positive national transformation. \n'),
 HumanMessage(content='what are his policies'),
 AIMessage(content='Sajith Premadasa\'s manifesto, "Win for All," focuses on five key points:\n\n* **Building a resilient economy:** This likely includes plans to address the economic crisis, promote growth, and create jobs.\n* **Empowering every citizen:** This coul

In [224]:
question = "what are ranil's education manifestos"
print(chatbot(question))

1111
queryType='search' query='education manifestos' candidates=1 person1='ranil_wickramasinghe' person2='null' person3='null'
Ranil Wickremesinghe's manifesto mentions modernizing school infrastructure to face the era of technology by 2040. Funding will be provided by the government. A program to equip teachers with new knowledge and training will be implemented in 2025. 



In [165]:
chat_history

[HumanMessage(content='who is ranil'),
 AIMessage(content=''),
 HumanMessage(content='who is ranil'),
 AIMessage(content='Ranil is a Sri Lankan politician who has held various ministerial positions under different governments. He is described as a "gleaming beacon" who has placed his "seal in the history of Sri Lanka". The document states that he is the one who brought change to Sri Lanka and made it a country that smiles. He is fighting against poverty, unemployment, and malnourishment. He is not fighting for a position, but for the lives of the youth and for a developed Sri Lanka. \n'),
 HumanMessage(content='what are his education manifestos'),
 AIMessage(content="Ranil's education manifesto focuses on:\n\n* **Recruiting 2500 new teachers** through a competitive examination.\n* **Providing special training** for these new teachers for 3 months before they are posted.\n* **Changing the method of compiling school textbooks** and revising the curricula to focus on interesting approache

In [155]:
chat_history

[HumanMessage(content='who is ranil'),
 AIMessage(content='Ranil Wickremesinghe is the current President of Sri Lanka. He is running for re-election in the upcoming presidential election. \n'),
 HumanMessage(content='what are his education manifestos'),
 AIMessage(content="Ranil Wickremesinghe's manifesto includes plans for:\n\n* **School education:**  The manifesto doesn't specify details, but it mentions school education as a priority.\n* **Vocational training:**  He plans to establish a new entity called Vocational Skills Sri Lanka (VSSL) in 2025, merging the National Apprentice and Industrial Training Authority and the Vocational Training Authority of Sri Lanka. This will bring all vocational training schools and centers under one umbrella.\n* **Degree awarding institutions:**  The manifesto mentions the ability to upgrade from these institutions to university level later, if necessary. \n")]

In [166]:
question = "what are his key policies"
print(chatbot(question))

1111
[SystemMessage(content='You have the ability to determine whether the user question is general, or it is related to a specific person or it is a comparison between multiple persons.'), HumanMessage(content='who is ranil'), AIMessage(content=''), HumanMessage(content='who is ranil'), AIMessage(content='Ranil is a Sri Lankan politician who has held various ministerial positions under different governments. He is described as a "gleaming beacon" who has placed his "seal in the history of Sri Lanka". The document states that he is the one who brought change to Sri Lanka and made it a country that smiles. He is fighting against poverty, unemployment, and malnourishment. He is not fighting for a position, but for the lives of the youth and for a developed Sri Lanka. \n'), HumanMessage(content='what are his education manifestos'), AIMessage(content="Ranil's education manifesto focuses on:\n\n* **Recruiting 2500 new teachers** through a competitive examination.\n* **Providing special trai

AttributeError: 'NoneType' object has no attribute 'queryType'

In [154]:
question = "who is sajith"
print(chatbot(question))

1111
[SystemMessage(content='You have the ability to determine whether the user question is general, or it is related to a specific person or it is a comparison between multiple persons.'), HumanMessage(content='who is ranil'), AIMessage(content='Ranil Wickremesinghe is the current President of Sri Lanka. He is running for re-election in the upcoming presidential election. \n'), HumanMessage(content='what are his education manifestos'), AIMessage(content="Ranil Wickremesinghe's manifesto includes plans for:\n\n* **School education:**  The manifesto doesn't specify details, but it mentions school education as a priority.\n* **Vocational training:**  He plans to establish a new entity called Vocational Skills Sri Lanka (VSSL) in 2025, merging the National Apprentice and Industrial Training Authority and the Vocational Training Authority of Sri Lanka. This will bring all vocational training schools and centers under one umbrella.\n* **Degree awarding institutions:**  The manifesto mention

AttributeError: 'NoneType' object has no attribute 'queryType'

In [106]:
question = "what are his education manifestos"
print(chatbot(question))

1111
[SystemMessage(content='use the chat history provided to analyze the query and convert the query into a standalone format which does not need chat history. You have the ability to determine whether the user question is general, or it is related to a specific person or it is a comparison between multiple persons.'), HumanMessage(content='who is sajith'), AIMessage(content='Sajith Premadasa is the Leader of the Samagi Jana Sandanya (SJS). This information is found on page 4 of the document "Sajith_manifesto_english.pdf". \n'), HumanMessage(content='what are his education manifestos'), AIMessage(content="Sajith Premadasa's education manifesto focuses on revamping the education sector to build internationally competitive human capital. Here are some key points:\n\n* **Transforming schools into smart schools:** This includes providing 100% electricity, water facilities, and physical resources to all schools in Sri Lanka.\n* **Addressing teacher issues:** This includes filling teacher v

AttributeError: 'NoneType' object has no attribute 'queryType'

In [99]:
question = "when sl election"
print(chatbot(question))

1111
[SystemMessage(content='You have the ability to determine whether the user question is general, or it is related to a specific person or it is a comparison between multiple persons. use the chat history provided to analyze the query'), HumanMessage(content='who is sajith'), AIMessage(content='Sajith Premadasa is the Leader of the Samagi Jana Sandanya (SJS) and the candidate for the election mentioned in the document. \n'), HumanMessage(content='what are his education manifestos'), AIMessage(content="The document states that the education sector will be revamped to build internationally competitive human capital. This means that Sajith Premadasa's education manifesto focuses on creating a system that produces graduates who can thrive in a rapidly changing world and contribute to the growth of Sri Lanka. \n"), HumanMessage(content='when sl election')]
queryType='general' query='when sl election' candidates=0 person1='null' person2='null' person3='null'
The Sri Lankan Presidential el

In [91]:
question = "who is sajith"
print(chatbot(question))

1111
[SystemMessage(content='You have the ability to determine whether the user question is general, or it is related to a specific person or it is a comparison between multiple persons. use the chat history provided to analyze the query'), HumanMessage(content='who is ranil'), AIMessage(content=''), HumanMessage(content='what are his education manifestos'), AIMessage(content="Ranil's education manifesto focuses on modernizing school infrastructure, equipping teachers with new knowledge and training, revising the curriculum to focus on soft skills, life skills, and moral values, and enabling students to start university education at the age of 17. \n"), HumanMessage(content='who is sajith')]
queryType='search' query='who is sajith' candidates=1 person1='sajith_premadasa' person2='null' person3='null'
Sajith Premadasa is the leader of the Samagi Jana Sandanaya (SJS) and the candidate for the Presidential election in the document provided. He is presented as a new political force with a 

In [92]:
chat_history

[HumanMessage(content='who is ranil'),
 AIMessage(content=''),
 HumanMessage(content='what are his education manifestos'),
 AIMessage(content="Ranil's education manifesto focuses on modernizing school infrastructure, equipping teachers with new knowledge and training, revising the curriculum to focus on soft skills, life skills, and moral values, and enabling students to start university education at the age of 17. \n"),
 HumanMessage(content='who is sajith'),
 AIMessage(content='Sajith Premadasa is the leader of the Samagi Jana Sandanaya (SJS) and the candidate for the Presidential election in the document provided. He is presented as a new political force with a vision to create a new Sri Lanka. \n')]

In [93]:
question = "what are his education manifestos"
print(chatbot(question))

1111
[SystemMessage(content='You have the ability to determine whether the user question is general, or it is related to a specific person or it is a comparison between multiple persons. use the chat history provided to analyze the query'), HumanMessage(content='who is ranil'), AIMessage(content=''), HumanMessage(content='what are his education manifestos'), AIMessage(content="Ranil's education manifesto focuses on modernizing school infrastructure, equipping teachers with new knowledge and training, revising the curriculum to focus on soft skills, life skills, and moral values, and enabling students to start university education at the age of 17. \n"), HumanMessage(content='who is sajith'), AIMessage(content='Sajith Premadasa is the leader of the Samagi Jana Sandanaya (SJS) and the candidate for the Presidential election in the document provided. He is presented as a new political force with a vision to create a new Sri Lanka. \n'), HumanMessage(content='what are his education manifes

AttributeError: 'NoneType' object has no attribute 'queryType'

In [62]:
chat_history

[HumanMessage(content='who is ranil'),
 AIMessage(content='Ranil is a leader who is described as someone who knows how to face challenges, finishes what he starts, and talks less and acts with precision. He is also described as someone who has brought positive change to Sri Lanka. \n')]

In [63]:
question = "what is his education manifestos"
print(chatbot(question))

1111
queryType='general' query='what is his education manifestos' candidates=0 person1='null' person2='null' person3='null'
Ranil Wickremesinghe's education manifesto is focused on "Operation – Beyond 2025, Towards a Prosperous Nation", which is one of the five core principles of his election manifesto.  The manifesto is titled ‘Five years of winning the country with Ranil.’  However, the provided context does not contain specific details about his education policies. 



In [36]:
question = "who is sajeev"
print(qa_chain.invoke(question).content)

1111
queryType='general' query='who is sajeev' candidates=0 person1='null' person2='null' person3='null'
This question is not related to the Sri Lankan election. As an election chatbot, I can't provide you with an answer to this. 



In [37]:
question = "when is sl election"
print(chatbot(question))

1111
queryType='general' query='when is sl election' candidates=0 person1='null' person2='null' person3='null'
The Sri Lankan Presidential election is scheduled for **September 21, 2024**. 



In [38]:
chat_history

[HumanMessage(content='when is sl election'),
 AIMessage(content='The Sri Lankan Presidential election is scheduled for **September 21, 2024**. \n')]

In [39]:
question = "who are the candidates"
print(chatbot(question))

1111
queryType='general' query='who are the candidates' candidates=0 person1='null' person2='null' person3='null'
The main candidates in the Sri Lankan Presidential election are:

* **Ranil Wickremesinghe:** The incumbent President and leader of the United National Party (UNP).
* **Sajith Premadasa:** The leader of the opposition and a member of the Samagi Jana Balawegaya (SJB).
* **Anura Kumara Dissanayake:** The leader of the National People's Power (NPP).
* **Namal Rajapaksa:** A member of the Sri Lanka Podujana Peramuna (SLPP) and a potential candidate. 

There are other candidates as well, including Nuwan Bopage, who is the Presidential candidate for the Alliance for People's Struggle. 



In [40]:
chat_history = []
question = "who are the candidates"
print(chatbot(question))

1111
queryType='general' query='who are the candidates' candidates=0 person1='null' person2='null' person3='null'
The main candidates in the Sri Lankan presidential election are:

* **Ranil Wickremesinghe:** The incumbent president and leader of the United National Party (UNP).
* **Sajith Premadasa:** The leader of the opposition and the candidate of the Samagi Jana Balawegaya (SJB).
* **Anura Kumara Dissanayake:** The candidate of the National People's Power (NPP).
* **Namal Rajapaksa:** The candidate of the Sri Lanka Podujana Peramuna (SLPP) and the son of former president Mahinda Rajapaksa.
* **Nuwan Bopage:** The presidential candidate for the Alliance for People's Struggle, which was established by the Aragalaya activists. 

There are also a number of other candidates running in the election. 



In [41]:
chat_history

[HumanMessage(content='who are the candidates'),
 AIMessage(content="The main candidates in the Sri Lankan presidential election are:\n\n* **Ranil Wickremesinghe:** The incumbent president and leader of the United National Party (UNP).\n* **Sajith Premadasa:** The leader of the opposition and the candidate of the Samagi Jana Balawegaya (SJB).\n* **Anura Kumara Dissanayake:** The candidate of the National People's Power (NPP).\n* **Namal Rajapaksa:** The candidate of the Sri Lanka Podujana Peramuna (SLPP) and the son of former president Mahinda Rajapaksa.\n* **Nuwan Bopage:** The presidential candidate for the Alliance for People's Struggle, which was established by the Aragalaya activists. \n\nThere are also a number of other candidates running in the election. \n")]

In [50]:
chat_history = []
question = "who is ranil"
print(chatbot(question))

1111
queryType='general' query='who is ranil wickramasinghe' candidates=0 person1='null' person2='null' person3='null'
Ranil Wickremesinghe is the current President of Sri Lanka. He is running for re-election in the upcoming presidential election on September 21st. 



In [43]:
chat_history

[HumanMessage(content='who is ranil'),
 AIMessage(content='Ranil Wickremesinghe is a Sri Lankan politician who is currently serving as the President of Sri Lanka. He is a prominent figure in Sri Lankan politics and has been involved in various roles, including Prime Minister. \n')]

In [52]:
chat_history

[HumanMessage(content='who is ranil'),
 AIMessage(content='Ranil Wickremesinghe is the current President of Sri Lanka. He is running for re-election in the upcoming presidential election on September 21st. \n')]

In [54]:
question = "what is his manifesto's about education"
print(chatbot(question))

1111


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.


InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting

In [2]:
question = "who is sajeev"
print(qa_chain.invoke(question).content)

This question is not related to the Sri Lankan election. As an election chatbot, I can't provide you with an answer to this. 



In [7]:
question = "who is the father of physics"
print(qa_chain.invoke(question).content)

This question is not related to Sri Lankan elections. As an election chatbot, I can't provide you with an answer to this. 



In [8]:
question = "when is sl election"
print(qa_chain.invoke(question).content)

The Sri Lankan presidential election is scheduled for **September 21, 2024**. 



In [9]:
question = "updates on sl election"
print(qa_chain.invoke(question).content)

The Sri Lankan Presidential election is scheduled for 2024.  Here are some updates:

* **Support for Ranil Wickremesinghe has surged:**  Recent polls show a significant increase in support for the incumbent President, Ranil Wickremesinghe.
* **Close race between Premadasa and Dissanayake:** Sajith Premadasa (SJB) and Anura Kumara Dissanayake (NPP/JVP) are currently leading the polls, neck and neck.
* **SLPP support remains low:** Support for a generic SLPP candidate remains unchanged at 4%, with Namal Rajapaksa only declaring his candidacy in August.
* **Party crossovers and changes in political loyalties:**  There has been an unprecedented trend of party crossovers and changes in political loyalties, with many SLPP MPs switching sides to support Wickremesinghe, Premadasa, or Dilith Jayaweera's Mawbima Janatha Party (MJP).
* **Manifestos released:**  Namal Rajapaksa has released his manifesto, "Namal Dekma," while Sajith Premadasa is expected to release his manifesto, "Win For All," so

In [3]:
question = "who is ranil"
print(qa_chain.invoke(question).content)

Ranil is a presidential candidate in the Sri Lankan election. He is being criticized for his age and some of his statements. Some people believe he is not fit to be president, while others support him and believe he has a plan for the future of the country. 



In [4]:
question = "sajith's manifestos about education"
print(qa_chain.invoke(question).content)

Sajith's manifesto focuses on revamping the education sector to build internationally competitive human capital.  Here are some key points:

* **Digital Learning:** Expanding the "Sakwala" program to provide digital learning platforms in schools, technical and vocational institutions, and universities.
* **Early Childhood Education (ECE):** Establishing a regulatory authority for ECE to develop responsible citizens.
* **Teacher Support:** Providing free public transportation to all teachers across Sri Lanka and addressing salary, pension, and service issues.
* **Smart Schools:** Transforming every school into a smart school by providing 100% electricity, water facilities, and physical resources.
* **STEEAM Education:** Prioritizing Science, Technology, English, Engineering, Arts, and Mathematics in all institutions of education.
* **University Reform:** Restructuring the University Grants Commission to address professional issues and strengthening the independence of the University.
* 

In [6]:
question = "compare education policies of sajith and anura"
print(qa_chain.invoke(question).content)

Both Sajith and Anura prioritize education in their manifestos, but their approaches differ in focus and implementation. 

**Sajith's Manifesto:**

* **Focus on accessibility and quality:** Sajith emphasizes making education accessible to all by providing free transportation for teachers, ensuring access to digital learning platforms, and upgrading infrastructure. He also aims to improve quality by reforming the Grade 5 Scholarship Exam, strengthening early childhood education, and promoting English language proficiency.
* **Emphasis on STEM and digital skills:** Sajith proposes a "STEEAM" education system to prioritize Science, Technology, Engineering, Arts, and Mathematics, and plans to expedite teacher training in these areas. He also emphasizes digital learning and the use of technology in education.
* **Specific initiatives:** Sajith outlines specific initiatives like the "Sakwala" program for digital learning, the "Bim Saviya" program for human capital development, and a monthly 